# Preprocess Monitoring Points

Adrian Wiegman | arhwiegman.github.io | adrian.wiegman@usda.gov

2023-03-30

---

In this notebook extract subwatershed ids for each monitoring within the [MEP study area](https://www.mass.gov/guides/the-massachusetts-estuaries-project-and-reports). 

input data is an excel table of points coordinates and watershed ids. 

the output will be a list of subwatershed ids with asociated point ids. 

Processing Steps


the fields in the Assess data will be used generate landcover classes for N loading
then the TaxPar LOC_ID's will be used to index the landcover classes for each discrete
the merged TaxPar layer will be intersected with watershed boundaries

In [45]:
# this codeblock sets up the environment from jupyter notebooks
setup_notebook = "C:/Users/Adrian.Wiegman/Documents/GitHub/Wiegman_USDA_ARS/MEP/_Setup.ipynb"
%run $setup_notebook # magic command to run the notebook 
fn_hello()

***
loading python modules...

  `module_list` contains names of all loaded modules

...module loading complete

***
loading user defined functions...

type `fn_`+TAB to for autocomplete suggestions

 the object `def_list` contains user defined function names:
   fn_get_info
   fn_arcgis_table_to_df
   fn_arcgis_table_to_np_to_pd_df
   fn_try_mkdir
   fn_hello
   fn_recursive_glob_search
   fn_regex_search_replace
   fn_regex_search_0
   fn_arcpy_table_to_excel
   fn_agg_sum_df_on_group
   fn_add_prefix_suffix_to_selected_cols
   fn_calc_pct_cover_within_groups

 use ??{insert fn name} to inspect
 for example running `??fn_get_info` returns:
Signature: fn_get_info(name='fn_get_info')
Source:   
def fn_get_info(name='fn_get_info'):
    '''
    returns the source information about a given function name
    '''
    get_ipython().run_line_magic('pinfo2', '$name')
File:      c:\workspace\geodata\mep\<ipython-input-1-28f96385f735>
Type:      function

 you can also use `fn_get_info(name={ins

In [46]:
# last run 6/27/2023
# read in data
fn_arcpy_table_to_excel('MEP_Monitoring_Sites_MeasOnly',odr,'MEP_Monitoring_Sites_MeasOnly_TableToExcel.xlsx')

In [47]:
inpath = os.path.join(odr,'MEP_Monitoring_Sites_MeasOnly_TableToExcel.xlsx')
df_in = pd.read_excel(inpath)
#df_in = fn_arcgis_table_to_df('MEP_Monitoring_Sites_MeasOnly').reset_index()
df_in.rename(columns={"OBJECTID":"FID"},inplace=True)
df_in.info()
df_in.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   FID         100 non-null    int64  
 1   Lat         100 non-null    float64
 2   Lon         100 non-null    float64
 3   Region      100 non-null    object 
 4   MEP         100 non-null    object 
 5   Site_Name   100 non-null    object 
 6   Region_MEP  100 non-null    object 
 7   SUBW_NAME   100 non-null    object 
 8   Notes_AW    12 non-null     object 
dtypes: float64(2), int64(1), object(6)
memory usage: 7.2+ KB


,FID,Lat,Lon,Region,MEP,Site_Name,Region_MEP,SUBW_NAME,Notes_AW
0,1,41.681859,-70.918844,Buzzards Bay,Acushnet,Acushnet River,Buzzards Bay > Acushnet > Acushnet River,UpperAcushnetRiver;NewBedfordReservoir,NaN
1,2,41.681859,-70.918844,Buzzards Bay,Acushnet,Acushnet River,Buzzards Bay > Acushnet > Acushnet River,UpperAcushnetRiver;NewBedfordReservoir,NaN
2,3,41.681859,-70.918844,Buzzards Bay,Acushnet,Acushnet River,Buzzards Bay > Acushnet > Acushnet River,UpperAcushnetRiver;NewBedfordReservoir,NaN
3,4,41.681859,-70.918844,Buzzards Bay,Acushnet,Acushnet River,Buzzards Bay > Acushnet > Acushnet River,UpperAcushnetRiver;NewBedfordReservoir,NaN
4,5,41.553741,-71.126612,Buzzards Bay,Westport,Adamsville Brook,Buzzards Bay > Westport > Adamsville Brook,AdamsvilleBrook,A portion of this watershed falls within Rhode...


In [48]:
#df['SUBW_NAME'].replace(r'^\s*$', np.nan, regex=True)
df = df_in.drop('Notes_AW',axis=1).dropna() # make a working copy
df['SUBW_NAME'].str.strip(to_strip = None) # remove whitespace
df.reset_index(inplace=True)
df.info()
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       100 non-null    int64  
 1   FID         100 non-null    int64  
 2   Lat         100 non-null    float64
 3   Lon         100 non-null    float64
 4   Region      100 non-null    object 
 5   MEP         100 non-null    object 
 6   Site_Name   100 non-null    object 
 7   Region_MEP  100 non-null    object 
 8   SUBW_NAME   100 non-null    object 
dtypes: float64(2), int64(2), object(5)
memory usage: 7.2+ KB
   index  FID        Lat        Lon        Region       MEP         Site_Name  \
0      0    1  41.681859 -70.918844  Buzzards Bay  Acushnet    Acushnet River   
1      1    2  41.681859 -70.918844  Buzzards Bay  Acushnet    Acushnet River   
2      2    3  41.681859 -70.918844  Buzzards Bay  Acushnet    Acushnet River   
3      3    4  41.681859 -70.918844  Buzzards Bay  Acushnet 

In [49]:
df.SUBW_NAME

0     UpperAcushnetRiver;NewBedfordReservoir
1     UpperAcushnetRiver;NewBedfordReservoir
2     UpperAcushnetRiver;NewBedfordReservoir
3     UpperAcushnetRiver;NewBedfordReservoir
4                            AdamsvilleBrook
                       ...                  
95                           PeasePointBrook
96                             TiasquamRiver
97                                BlackBrook
98                     UpperMashapaquitCreek
99    UpperAcushnetRiver;NewBedfordReservoir
Name: SUBW_NAME, Length: 100, dtype: object

In [50]:
SUBW_NAME = [] # name of subwatershed in MEP data
PROPORTION = [] # proportion of flow draining to monitoring site at corresponding FID
FID = [] # FID of monitoring site
for i in range(len(df.FID)):
    print(i)
    #print(i,"|",df.Region_MEP[i])
    S = df.SUBW_NAME[i].split(";") # split column SUBW_NAME on semicolen
    #print(S)
    for s in S:
        match = fn_regex_search_0(s,"\(.*\)") # find parentheses
        replaced = fn_regex_search_replace(s,"\(.*\)","") # replace parentheses with blank

        digit = fn_regex_search_0(s,"\d+") # extract numeric digits
        if digit == 'NA':
            p = 1
        else:
            p = int(digit)/100 # calculate proportion of flow from digits
        print("  ",s,"|",replaced,"|",p)
        
        if fn_regex_search_0(replaced,"\w+") == 'NA':
            print("      empty... skipping to next")
            next # skip columns that dont have match
        
        # append values to lists
        SUBW_NAME.append(replaced)
        PROPORTION.append(p)
        FID.append(df.FID[i])

0
   UpperAcushnetRiver | UpperAcushnetRiver | 1
   NewBedfordReservoir | NewBedfordReservoir | 1
1
   UpperAcushnetRiver | UpperAcushnetRiver | 1
   NewBedfordReservoir | NewBedfordReservoir | 1
2
   UpperAcushnetRiver | UpperAcushnetRiver | 1
   NewBedfordReservoir | NewBedfordReservoir | 1
3
   UpperAcushnetRiver | UpperAcushnetRiver | 1
   NewBedfordReservoir | NewBedfordReservoir | 1
4
   AdamsvilleBrook | AdamsvilleBrook | 1
5
   MillPond | MillPond | 1
   ChargePond(50%) | ChargePond | 0.5
   EastHeadPond(50%) | EastHeadPond | 0.5
   LittleLongPond | LittleLongPond | 1
   EastBranchN | EastBranchN | 1
   FiveMilePond | FiveMilePond | 1
   AbnerPond | AbnerPond | 1
   FearingPond | FearingPond | 1
   NewLongPond | NewLongPond | 1
   ThreeCorneredPond | ThreeCorneredPond | 1
   AgawamReservoirS | AgawamReservoirS | 1
   NewGrassyPond | NewGrassyPond | 1
   FawnPond | FawnPond | 1
   BumpsPond | BumpsPond | 1
   CollegePond | CollegePond | 1
   HalfwayPond | HalfwayPond | 1
   Agaw

In [51]:
# create output dataframe
print(len(FID),
len(SUBW_NAME),
len(PROPORTION))
_ = pd.DataFrame({'FID':FID,
                       'SUBW_NAME':SUBW_NAME,
                       'PROPORTION':PROPORTION})
print(_)
print(df[['FID','Lat','Lon','Region_MEP']])
df_out = _.merge(df[['FID','Lat','Lon','Region_MEP']],on='FID')
print(df_out)
df_out.to_csv(os.path.join(odr,"df_monitoring_point_subs.csv"))

252 252 252
     FID              SUBW_NAME  PROPORTION
0      1     UpperAcushnetRiver         1.0
1      1    NewBedfordReservoir         1.0
2      2     UpperAcushnetRiver         1.0
3      2    NewBedfordReservoir         1.0
4      3     UpperAcushnetRiver         1.0
..   ...                    ...         ...
247   97          TiasquamRiver         1.0
248   98             BlackBrook         1.0
249   99  UpperMashapaquitCreek         1.0
250  100     UpperAcushnetRiver         1.0
251  100    NewBedfordReservoir         1.0

[252 rows x 3 columns]
    FID        Lat        Lon                                   Region_MEP
0     1  41.681859 -70.918844     Buzzards Bay > Acushnet > Acushnet River
1     2  41.681859 -70.918844     Buzzards Bay > Acushnet > Acushnet River
2     3  41.681859 -70.918844     Buzzards Bay > Acushnet > Acushnet River
3     4  41.681859 -70.918844     Buzzards Bay > Acushnet > Acushnet River
4     5  41.553741 -71.126612   Buzzards Bay > Westport > Ada